## Analyze the number of harmonics

For this it is important that the controller did not control the tip deflection
so that all simulations have the exact same tip deflection, so it should also be
run without turbulence.

In [ ]:
import glob
import os
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from weis.visualization.utils import load_OMsql_multi
from rosco.toolbox.ofTools.fast_io import output_processing
from cmcrameri import cm

plt.style.use("torque.mplstyle")

%matplotlib widget

In [ ]:
def interpolate_at_azimuth(azimuth, signal, target_azimuth):
    """Interpolate signal at specific azimuth angle for each rotor revolution."""
    azimuth = np.asarray(azimuth)
    signal = np.asarray(signal)
    interpolated_values = []

    for i in range(len(azimuth) - 1):
        az_curr = azimuth[i]
        az_next = azimuth[i + 1]

        if az_curr < target_azimuth <= az_next:
            fraction = (target_azimuth - az_curr) / (az_next - az_curr)
            interp_value = signal[i] + fraction * (signal[i + 1] - signal[i])
            interpolated_values.append(interp_value)

    return interpolated_values

## Load Data

In [ ]:
# Paths
base_path = "../../../data/torque/harmonics/openfast_runs"
log_path = "../../../data/torque/harmonics/log_harmonics.sql*"

# Load metadata (rank -> n_harmonics mapping)
print("Loading metadata...")
data_dict = load_OMsql_multi(log_path)
df_meta = pd.DataFrame(
    {
        "rank": data_dict["rank"],
        "iter": data_dict["iter"],
        "n_harmonics": data_dict["tune_rosco_ivc.TCIPC_nHarmonics"],
    }
)
print(f"Available n_harmonics: {sorted(df_meta['n_harmonics'].unique())}")

In [ ]:
# Load OpenFAST output files (.outb)
print("\nLoading OpenFAST outputs...")
outb_files = sorted(glob.glob(os.path.join(base_path, "rank_*/*.outb")))
op_openfast = output_processing.output_processing()
openfast_data = op_openfast.load_fast_out(outb_files, tmin=0, verbose=False)
print(f"Loaded {len(openfast_data)} OpenFAST files")

In [ ]:
# Load ROSCO debug files (.RO.dbg)
print("\nLoading ROSCO debug files...")
dbg_files = sorted(glob.glob(os.path.join(base_path, "rank_*/*.RO.dbg")))
op_rosco = output_processing.output_processing()
rosco_data = op_rosco.load_fast_out(dbg_files, tmin=0, verbose=False)
print(f"Loaded {len(rosco_data)} ROSCO debug files")

# Verify TipDxc_TowerPas exists
if "TipDxc_TowerPas" in rosco_data[0]["meta"]["channels"]:
    print("✓ TipDxc_TowerPas column found")
else:
    print("✗ WARNING: TipDxc_TowerPas column not found!")

## Step 1: Compute Ground Truth (Single Value)

Calculate mean tower-passing deflection across all blades and all simulations.

In [ ]:
# Tower passing azimuths
tower_azimuths = {1: 180.0, 2: 60.0, 3: 300.0}

# Collect all tower-passing deflection means
all_tower_passing_means = []

for openfast_dict in openfast_data:
    azimuth = openfast_dict["Azimuth"]
    tipdxc1 = openfast_dict["TipDxc1"]
    tipdxc2 = openfast_dict["TipDxc2"]
    tipdxc3 = openfast_dict["TipDxc3"]

    # Interpolate each blade at its tower-passing azimuth
    blade1_tp = interpolate_at_azimuth(azimuth, tipdxc1, tower_azimuths[1])
    blade2_tp = interpolate_at_azimuth(azimuth, tipdxc2, tower_azimuths[2])
    blade3_tp = interpolate_at_azimuth(azimuth, tipdxc3, tower_azimuths[3])

    # Mean of three blades for each tower-passing event
    n_events = min(len(blade1_tp), len(blade2_tp), len(blade3_tp))
    for j in range(n_events):
        event_mean = (blade1_tp[j] + blade2_tp[j] + blade3_tp[j]) / 3.0
        all_tower_passing_means.append(event_mean)

# Overall ground truth
ground_truth = np.mean(all_tower_passing_means)
ground_truth_std = np.std(all_tower_passing_means)

print(f"\n{'=' * 60}")
print(f"GROUND TRUTH: {ground_truth:.6f} m")
print(f"Std deviation: {ground_truth_std:.6f} m")
print(f"Tower-passing events: {len(all_tower_passing_means)}")
print(f"{'=' * 60}")

## Step 2: Build Comparison DataFrame

Extract all controller estimates and compute errors (time > 300s only).

In [ ]:
data_rows = []

for rosco_dict in rosco_data:
    # Extract rank from filename
    filename = rosco_dict["meta"]["filename"]
    match = re.search(r"rank_(\d+)", filename)
    if not match:
        continue
    rank = int(match.group(1))

    # Get n_harmonics for this rank
    meta_subset = df_meta[(df_meta["rank"] == rank) & (df_meta["iter"] == 0)]
    if len(meta_subset) == 0:
        continue
    n_harmonics = int(meta_subset["n_harmonics"].values[0])

    # Extract all controller estimates (only time > 300s)
    time = rosco_dict["Time"]
    controller_estimate = rosco_dict["TipDxc_TowerPas"]

    # Create rows for time samples > 300s
    for t, est in zip(time, controller_estimate):
        if t > 300:
            data_rows.append(
                {
                    "rank": rank,
                    "n_harmonics": n_harmonics,
                    "time": t,
                    "controller_estimate": est,
                    "ground_truth": ground_truth,
                    "error": ground_truth - est,
                }
            )

df = pd.DataFrame(data_rows)
print(f"\nDataFrame shape: {df.shape}")
print(f"Time samples per simulation: ~{len(df) // len(rosco_data)}")
print(f"\nError summary by n_harmonics:")
print(df.groupby("n_harmonics")["error"].describe())

## Visualizations

In [ ]:
# Time-domain plot
fig, ax = plt.subplots()

for n_harm in sorted(df["n_harmonics"].unique()):
    df_sub = df[df["n_harmonics"] == n_harm]
    ax.plot(
        df_sub["time"],
        df_sub["error"],
        alpha=0.7,
        linewidth=0.8,
        label=f"n={int(n_harm)}",
    )

ax.axhline(0, color="k", linestyle="--", linewidth=1, zorder=1)
ax.set_xlabel("Time (s)")
ax.set_ylabel("Error (m)\n[ground truth - controller estimate]")
ax.set_title(f"Controller Estimation Error (Ground Truth = {ground_truth:.3f} m)")
ax.grid(alpha=0.3)
ax.legend(title="n_harmonics", bbox_to_anchor=(1.05, 1), loc="upper left")

plt.tight_layout()
plt.show()

In [ ]:
# Violin plot with mean markers
fig, ax = plt.subplots()

sns.violinplot(
    data=df,
    x="n_harmonics",
    y="error",
    ax=ax,
    palette="cmc.batlow",
    inner=None,
)

# Add mean values as circles
mean_errors = df.groupby("n_harmonics")["error"].mean()
ax.scatter(
    range(len(mean_errors)),
    mean_errors.values,
    color="black",
    marker="x",
    s=30,
    zorder=10,
    label="Mean",
)

ax.axhline(0, color="k", linestyle="--", linewidth=1, zorder=10)
ax.set_axisbelow(True)
ax.set_xlabel("Number of harmonics (-)")
ax.set_ylabel("Estimation error (m)")
ax.grid(axis="y")
ax.legend()
ax.set_ylim((-1.5, 1.5))

plt.tight_layout()
plt.show()

plt.savefig("../figures/n_harmonics.pdf")

In [ ]:
# PDF plot (probability density function).
from scipy.stats import gaussian_kde

fig, ax = plt.subplots()

# Global x-range from all errors
global_errors = df["error"].values
x_min, x_max = np.min(global_errors), np.max(global_errors)
x_range = np.linspace(x_min, x_max, 200)

# Colormap
n_unique = len(df["n_harmonics"].unique())
colors = cm.batlow(np.linspace(0, 1, n_unique))

for idx, n_harm in enumerate(sorted(df["n_harmonics"].unique())):
    df_sub = df[df["n_harmonics"] == n_harm]
    errors = df_sub["error"].values

    # Compute KDE over shared x_range
    kde = gaussian_kde(errors)
    density = kde(x_range)

    ax.plot(
        x_range,
        density,
        linewidth=2,
        label=f"n={int(n_harm)}",
        color=colors[idx],
    )

ax.axvline(0, color="k", linestyle="--", linewidth=1, alpha=0.5, label="Perfect")
ax.set_xlabel("Error (m) [ground truth - controller estimate]")
ax.set_ylabel("Normalized Density")
ax.set_title(f"Error Distribution by n_harmonics (Ground Truth = {ground_truth:.3f} m)")
ax.grid(alpha=0.3, axis="y")
ax.legend(title="n_harmonics")

plt.tight_layout()
plt.show()
